# Тетрадка с примерами для презентации

In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData, OpenCDP
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding, IndexItemEmbeddings, CategoricalItemEmbeddings, SVDItemEmbeddings, MixedEmbeddings
from src.models import ResponseModel

device = 'cuda:0'
seed = 7331
pkl_path = '../pkl/'

### Вынесение split датасета за пределы core-pipeline
Вот это происходит не в основном пайплайне, два датасета собраны из разных файлов.

In [2]:
!ls ../data/recsys/OpenCDP/cosmetics/

2019-Dec.csv  2019-Nov.csv  2019-Oct.csv  2020-Feb.csv	2020-Jan.csv


In [3]:
# train_dataset = OpenCDP('../data/recsys/OpenCDP/cosmetics/', files=['2019-Dec.csv'])
train_dataset = OpenCDP.load(os.path.join(pkl_path, 'ocdp_train.pkl'))

Нужно только явно указать индексацию айтемов, чтобы было обученные эмбеддинги как-то соотносились с тестовым множеством.

In [4]:
# test_dataset = OpenCDP(
#     '../data/recsys/OpenCDP/cosmetics/', 
#     files=['2020-Jan.csv'], 
#     item_id2index = train_dataset.item_id2index
# )
test_dataset =OpenCDP.load(os.path.join(pkl_path, 'ocdp_test.pkl'))

# Добавление возможности передачи признаков для пользователя/айтема/отдельных действий в сессии, включая категориальные

In [5]:
index_embeddings = IndexItemEmbeddings(train_dataset.n_items, embedding_dim = 2)
category_embeddings = CategoricalItemEmbeddings(
    train_dataset.item_categorical
)
svd_embeddings = SVDItemEmbeddings(train_dataset.user_item_matrix, embedding_dim=8)

mixed_embeddings = MixedEmbeddings(
    index_embeddings,
    svd_embeddings,
    category_embeddings
)

### Fit датасета на части данных с последующим transform дополнительных кусочков 

In [6]:
model = ResponseModel('LogisticRegression', mixed_embeddings)
model.fit(train_dataset, batch_size=256, num_epochs=2)
test_dataset = model.transform(test_dataset, batch_size=256)

train:   0%|          | 0/2 [00:00<?, ?it/s]

transform...:   0%|          | 0/96 [00:00<?, ?it/s]

### Возврат результатов работы модели отклика в формате, пригодном для последующего подсчета бизнес-метрик 

что-то около: session_id, user_id, item_id,  <колонки из исходного датафрейма типа timestamp и значений отдельных признаков>, response, response_proba)

In [7]:
df = test_dataset.to_iteraction_matrix()
df

,recommendation_idx,item_feature_idx,session_id,user_id,timestamp,user_feature_idx,user_idx,response,predicted_prob,predicted_response,item_categorical
0,0,0,00040372-9fc7-4759-b96e-050e7a8f5c99,598956005,2020-01-09 18:39:29 UTC,0,13419,0,0.504724,1,"[0, 0]"
1,0,0,00040372-9fc7-4759-b96e-050e7a8f5c99,598956005,2020-01-09 18:39:29 UTC,0,13419,0,0.466401,1,"[0, 127]"
2,0,0,00040372-9fc7-4759-b96e-050e7a8f5c99,598956005,2020-01-09 18:39:29 UTC,0,13419,0,0.425856,1,"[0, 158]"
3,0,0,00040372-9fc7-4759-b96e-050e7a8f5c99,598956005,2020-01-09 18:39:29 UTC,0,13419,0,0.479871,1,"[0, 127]"
4,0,0,00040372-9fc7-4759-b96e-050e7a8f5c99,598956005,2020-01-09 18:39:29 UTC,0,13419,0,0.466401,1,"[0, 127]"
...,...,...,...,...,...,...,...,...,...,...,...
767465,96690,96690,fffd89d8-4a5d-42a6-8f63-29764a959d43,568871996,2020-01-31 05:13:12 UTC,96690,9193,1,0.582997,1,"[0, 0]"
767466,96690,96690,fffd89d8-4a5d-42a6-8f63-29764a959d43,568871996,2020-01-31 05:13:12 UTC,96690,9193,1,0.582997,1,"[0, 0]"
767467,96690,96690,fffd89d8-4a5d-42a6-8f63-29764a959d43,568871996,2020-01-31 05:13:12 UTC,96690,9193,1,0.582997,1,"[0, 0]"
767468,96690,96690,fffd89d8-4a5d-42a6-8f63-29764a959d43,568871996,2020-01-31 05:13:12 UTC,96690,9193,1,0.582997,1,"[0, 0]"
